# 05 - resumo de venda/faturamento - fechamento anual


In [1]:
''' mes-ano / vendas / faturado 
total / .../ '''

' mes-ano / vendas / faturado \ntotal / .../ '

In [1]:
import os,sys
current_dir = os.getcwd()

# muda o diretorio para o projeto --
# isso é necessario para que o python consiga importar os modulos do projeto
# caso contrario, ele nao consegue achar os modulos
project_root = os.path.join(current_dir, '..')
sys.path.append(project_root)
import pandas as pd
from Scripts.Reads import ReadRaw, ReadConfigs
from Scripts.Padronizes import PadronizeProdutos
os.chdir("..")

ano, mes = ReadConfigs()

# -- 1 - defines as bases de dados e trata certas colunas
df_nfs, _ = ReadRaw(['sh-nfs_final.xlsx'])
df_nfs = df_nfs[['Vl Total Nota', 'Dt. emissão']]  # seleciona só as colunas necessárias

df_pedidos, _ = ReadRaw(['sh-pedidos_final.xlsx'])
df_pedidos = df_pedidos[['Vl venda', 'Dt. emissão']]

# -- 4 - trandorma em mes-ano
df_pedidos['Dt. emissão'] = df_pedidos['Dt. emissão'].dt.to_period('M')
df_nfs['Dt. emissão'] = df_nfs['Dt. emissão'].dt.to_period('M')

# -- 5 - Agrupa por mês e soma os valores
vendas_mensal = df_pedidos.groupby('Dt. emissão')['Vl venda'].sum().reset_index()
faturado_mensal = df_nfs.groupby('Dt. emissão')['Vl Total Nota'].sum().reset_index()

# -- 7 -  mergga os dois DataFrames pela coluna 'Dt. emissão'
df_raw_mes_ano = pd.merge(
    vendas_mensal,
    faturado_mensal,
    on='Dt. emissão',
    how='outer'  # inclui meses que têm apenas vendas ou apenas faturamento
).fillna(0)

# -- 9 - mud o nome ds colunas
df_raw_mes_ano.columns = ['mes-ano', 'venda', 'faturado']

total_mes_faturado = df_raw_mes_ano['faturado'].sum()
total_mes_venda = df_raw_mes_ano['venda'].sum()

print(f'2024 - total:\n faturado | venda \n {total_mes_faturado} | {total_mes_venda} ')


import os
import pandas as pd

def salvar_relatorio_fechamento(df_raw_mes_ano):
    # Diretório onde o arquivo final será salvo
    base_path = f'resource/fechamento-anual/{ano}'
    nome_arquivo = 'fechamento-anual.xlsx'
    colunas = ['mes-ano', 'venda', 'faturado', 'ultimo']
    
    # Cria nova linha (filtra apenas o mês atual do df)
    mes_str = f'{ano}-{mes:02d}'
    linha_nova = df_raw_mes_ano[df_raw_mes_ano['mes-ano'] == mes_str]
    if linha_nova.empty:
        raise ValueError(f"Nenhum dado encontrado para {mes_str} no df_raw_mes_ano.")
    
    nova_linha = linha_nova.copy()
    nova_linha['ultimo'] = True

    # Caminho da pasta do mês anterior
    mes_anterior = mes - 1
    df_final = None
    if mes_anterior >= 1:
        pasta_anterior = os.path.join(base_path, f'{mes_anterior:02d}')
        arquivo_anterior = os.path.join(pasta_anterior, nome_arquivo)

        if os.path.exists(arquivo_anterior):
            df_ant = pd.read_excel(arquivo_anterior)
            if 'ultimo' in df_ant.columns:
                df_ant['ultimo'] = df_ant['ultimo'].fillna(False)
                idx_true = df_ant[df_ant['ultimo'] == True].index
                if not idx_true.empty:
                    df_ant.loc[idx_true, 'ultimo'] = False
            df_final = pd.concat([df_ant, nova_linha], ignore_index=True)

    # Se não tem mês anterior ou arquivo não existe
    if df_final is None:
        df_final = pd.DataFrame(columns=colunas)
        df_final = pd.concat([df_final, nova_linha], ignore_index=True)

    # Criar pasta do mês atual
    pasta_atual = os.path.join(base_path, f'{mes:02d}')
    os.makedirs(pasta_atual, exist_ok=True)
    caminho_saida = os.path.join(pasta_atual, nome_arquivo)

    # Escrever com xlsxwriter
    with pd.ExcelWriter(caminho_saida, engine='xlsxwriter') as writer:
        df_final.to_excel(writer, index=False, sheet_name='FECHAMENTO')
        
        worksheet = writer.sheets['FECHAMENTO']
        total_row = len(df_final) + 1  # +1 cabeçalho

    print(f"✅ Relatório salvo em: {caminho_saida}")
salvar_relatorio_fechamento(df_raw_mes_ano)

2024 - total:
 faturado | venda 
 139288.42 | 135198.7 
✅ Relatório salvo em: resource/fechamento-anual/2025\06\fechamento-anual.xlsx


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13020\3488145195.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, nova_linha], ignore_index=True)
